In [ ]:
import gzip
import math
import pickle

import seaborn as sns

import matplotlib.pyplot as plt
from rdkit.Chem import PandasTools, Crippen


import warnings
import numpy as np 
import pandas as pd
from rdkit import Chem
from rdkit import rdBase
from rdkit import DataStructs
rdBase.DisableLog('rdApp.error')
warnings.filterwarnings("ignore")

import argparse
from rdkit.Chem import PandasTools
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
import os
from mol_metrics import *
import logging
from rdkit.Chem import PandasTools, QED, Descriptors, rdMolDescriptors
import re

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt


In [ ]:
def calculate_properties(smiles_list):
    """
    Calculate QED, solubility, and SA scores for a list of SMILES strings.
    Args:
        smiles_list (list): List of SMILES strings.
    Returns:
        pd.DataFrame: DataFrame with columns ['QED', 'Solubility', 'SA'].
    """
    mols = pd.Series(smiles_list).apply(Chem.MolFromSmiles).dropna()
    mols = mols[mols.apply(lambda x: x.GetNumAtoms() > 1)]
    
    if mols.empty:
        return pd.DataFrame(columns=['SMILES', 'QED', 'Solubility', 'SA'])
    
    
    mols_df = pd.DataFrame({
        'SMILES': mols.apply(Chem.MolToSmiles),
        'QED': mols.apply(QED.qed),
        'Solubility': mols.apply(solubility),
        'SA': mols.apply(SA)
    })
    
    return mols_df

In [ ]:
def compute_top_k_means(properties_df, k):
    """
    Compute the mean of the top `k` values for each property in the DataFrame.
    Args:
        properties_df (pd.DataFrame): DataFrame with chemical properties.
        k (int): Number of top entries to consider.
    Returns:
        dict: Dictionary with mean values for each property.
    """
    if properties_df.empty or k <= 0:
        return {'Top_k_Mean_QED': 0, 'Top_k_Mean_Solubility': 0, 'Top_k_Mean_SA': 0}

    top_k_means = {}
    for col in ['QED', 'Solubility', 'SA']:
        top_k_means[f'Top_k_Mean_{col}'] = properties_df[col].nlargest(k).mean()
    
    return top_k_means

In [ ]:
def get_top_1_molecules(properties_df):
    top_molecules = {}
    for col in ['QED', 'Solubility', 'SA']:
        top_molecule = properties_df.nlargest(1, col)
        smile = top_molecule['SMILES'].values[0]
        value = top_molecule[col].values[0]
        top_molecules[col] = (smile, value)
    return top_molecules

In [ ]:
def visualize_combined_top_molecules(single_properties_list, multi_properties):
    """
    Visualize the top-1 molecules for single-property and multi-property optimized datasets and save the figure.
    Args:
        single_properties_list (list): List of properties dataframes for single-property optimized datasets.
        multi_properties (pd.DataFrame): Properties dataframe for multi-property optimized dataset.
        save_path (str): Path to save the figure.
    """
    # Create a grid layout: 4 rows (2 for images, 2 for labels) x 3 columns
    fig, axs = plt.subplots(4, 3, figsize=(18, 24))

    properties = ['QED', 'Solubility', 'SA']
    labels = ['QED (single)', 'Solubility (single)', 'SA (single)', 
              'QED (multi)', 'Solubility (multi)', 'SA (multi)']

  # Hide the label axes and set labels
    for i in range(3):
        axs[1, i].axis('off')  # Hide the second row used for labels of the first row
        axs[1, i].text(0.5, 0.5, labels[i], ha='center', va='center', fontsize=25)
        
        axs[3, i].axis('off')  # Hide the fourth row used for labels of the third row
        axs[3, i].text(0.5, 0.5, labels[i + 3], ha='center', va='center', fontsize=25)

    
    # Display top-1 molecules for single-property datasets
    print('Single Property:')
    for i, properties_df in enumerate(single_properties_list):
        top_molecules = get_top_1_molecules(properties_df)
        mol = Chem.MolFromSmiles(top_molecules[properties[i]][0])
        print(f'Single {properties[i]} Top 1 SMILES:', top_molecules[properties[i]][0], 'QED:', QED.qed(mol), 'Solubility:', solubility(mol), 'SA:', SA(mol))
        img = Draw.MolToImage(mol, size=(300, 300))
        axs[0, i].imshow(img)
        axs[0, i].axis('off')

    # Display top-1 molecules for multi-property optimized dataset
    print('Multi Properties:')
    top_molecules = get_top_1_molecules(multi_properties)
    for i, prop in enumerate(properties):
        mol = Chem.MolFromSmiles(top_molecules[prop][0])
        print(f'Multi {prop} Top 1 SMILES:', top_molecules[prop][0],'QED:', QED.qed(mol), 'Solubility:', solubility(mol), 'SA:', SA(mol))
        img = Draw.MolToImage(mol, size=(300, 300))
        axs[2, i].imshow(img)
        axs[2, i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
data = []
with open('zinc250k.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split(",")[1])

# Training canonical SMILES
training_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(sm)) for sm in data]
training_smiles = list(set(training_smiles))



In [ ]:

# Compute top k means for multiple k values
k_values = [1, 5, 10, 100, 1000, 2000, 4000, 6000, 8000] 

In [ ]:
def load_smiles_from_csv(file_path, training_smiles):
    generated_smiles = []
    with open(file_path, "r") as f:
        for line in f.readlines()[1:]:
            generated_smiles.append(line.strip())

    # SMILES to molecules
    generated_mols = np.array([Chem.MolFromSmiles(s) for s in generated_smiles if len(s.strip())])

    valid_smiles = [Chem.MolToSmiles(mol) for mol in generated_mols if mol != None and mol.GetNumAtoms() > 1 and Chem.MolToSmiles(mol) != ' ']
    unique_smiles = list(set(valid_smiles))
    novel_smiles = [smi for smi in unique_smiles if smi not in training_smiles]

    return novel_smiles

In [ ]:
qed_path = "QED/Results/QED/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_QED_generated_data.csv"
solubility_path = "Solubility/Results/Solubility/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_Solubility_generated_data.csv"
sa_path = "SA/Results/SA/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_SA_generated_data.csv"
all_path = "ALL/Results/ALL/ew=0.022_gw=1e-05/Alphas/0.5/logs/best_ALL_generated_data.csv"


# Load and process datasets
qed_smiles = load_smiles_from_csv(qed_path, training_smiles)
solubility_smiles = load_smiles_from_csv(solubility_path, training_smiles)
sa_smiles = load_smiles_from_csv(sa_path, training_smiles)
all_smiles = load_smiles_from_csv(all_path, training_smiles)


qed_properties = calculate_properties(qed_smiles)
solubility_properties = calculate_properties(solubility_smiles)
sa_properties = calculate_properties(sa_smiles)
all_properties = calculate_properties(all_smiles)



In [ ]:
print('QED:')

for k in k_values:
    top_k = compute_top_k_means(qed_properties, k)
    print(f"Single QED Top {k} Means: {top_k}")


In [ ]:
print('Solubility:')


for k in k_values:
    top_k = compute_top_k_means(solubility_properties, k)
    print(f"Single solubility Top {k} Means: {top_k}")


In [ ]:
print('SA:')


for k in k_values:
    top_k = compute_top_k_means(sa_properties, k)
    print(f"Single SA Top {k} Means: {top_k}")

In [ ]:
print('Multi Properties:')


for k in k_values:
    top_k = compute_top_k_means(all_properties, k)
    print(f"Multi Top {k} Means: {top_k}")

In [ ]:

# List of properties DataFrames for single-property datasets
single_properties_list = [qed_properties, solubility_properties, sa_properties]



